# Python ML Hands-On - Part 1

Dieses Notebook gehört zum Vortrag "Python ML Hands-On" an den Noser E-Days.

Ziel des Notebooks ist die Demonstration der Datenverarbeitung mit Pandas. Als Grundlage wird eine auf der Website vom BFS verfügbare Statistik zum Durchschnittlichen Mietpreis von Wohnungen verwendet. ([Link](https://www.bfs.admin.ch/bfs/de/home/statistiken/bau-wohnungswesen.assetdetail.15104263.html))


In [2]:
# Importieren der benötigten Bibliotheken (module)
import pandas as pd
import numpy as np

Das vom BFS runtergeladene Excel besteht aus mehreren Sheets, hier wird zu Demo nur auf das erste Sheet zugegriffen. Natürlich könnte der Code ohne weiteres auf mehrere Sheets erweitert werden.
In diesem Sheet sind die Daten aus dem Jahr 2019 erfasst, wobei die eigentliche Tabelle erst in Zeile 3 beginnt und die letzten 11 Zeilen nur noch Kommentare sind. In der ersten Spalte stehen die Kantonsnamen, diese wird deshalb als Index verwendet.

In [4]:
data_path = './Mietpreis.xlsx'
excel_file = pd.ExcelFile(data_path)
name_of_sheet1 = excel_file.sheet_names[0]
SKIPHEADER = 3  # die ersten 3 Zeilen ignorieren
SKIPFOOTER = 11  # die letzten 11 Zeilen ignorieren
INDEX_COL = 0  # Die Kantonsnamen
data = pd.read_excel(excel_file, name_of_sheet1, skiprows=SKIPHEADER, index_col=INDEX_COL, skipfooter=SKIPFOOTER)
data.head()  # oberste Zeilen ausgeben

,Total,Unnamed: 2,1,Unnamed: 4,2,Unnamed: 6,3,Unnamed: 8,4,Unnamed: 10,5,Unnamed: 12,6+,Unnamed: 14
NaN,Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF),Durch-schnittlicher Mietpreis,Vertrauens-intervall : \n± (in CHF)
Schweiz,1362,3,794,8,1101,5,1317,4,1574,5,1910,13,2360,41
Zürich,1592,8,951,20,1339,15,1538,12,1835,15,2253,36,2907,115
Bern,1223,8,702,19,988,13,1187,10,1417,13,1691,31,2079,117
Luzern,1368,9,757,31,1073,17,1315,14,1509,14,1849,36,2145,101


Der Index (die Zeilenbezeichnung) ist der jeweilige Kanton, also wird er auch so benannt:

In [3]:
data.index.name = 'Kanton'

In der Tabelle `data` stehen die Informationen über den durchschnittlichen Mietpreis, die Zeilen sind die Information pro Kanton, die Spalten sind die Anzahl Zimmer. Dabei gibt es pro Zimmer-Anzahl zwei Spalten, die eine mit dem durchschnittlichen Mietpreis, die andere mit dem Vertraunsintervall.
In einem ersten Schritt werden die Spalten mit dem Vertrauensintervall gelöscht, und die erste Zeile der Daten (die eigentlich ebenfalls zum Header gehört) entfernt:

In [4]:

data = data.drop(columns=[c for c in data.columns if 'Unnamed' in str(c)])  # Löschen der Spalten mit Unnamed im Titel, also den Vertrauensintervall-Spalten
data = data.iloc[1:]  # Löschen der ersten Zeile
data.head()


,Total,1,2,3,4,5,6+
Kanton,,,,,,,
Schweiz,1362,794,1101,1317,1574,1910,2360
Zürich,1592,951,1339,1538,1835,2253,2907
Bern,1223,702,988,1187,1417,1691,2079
Luzern,1368,757,1073,1315,1509,1849,2145
Uri,1210,X,846,1166,1329,1541,1745


Die Tabelle sieht schon sehr viel sauberer aus. Als Nächstes sollen die unbekannten Einträge, die Aktuell mit einem `X` gekennzeichnet sind, mit `np.nan` ersetzt werden.
Beim Ausgeben des Eintrags in der Spalte `1` für den Kanton Uri fällt auf, dass da nicht nur ein `X` drin steht, sondern auch noch Leerzeichen:

In [5]:
data.loc['Uri', 1]  # Zugriff auf Zeile 'Uri', Spalte "1" (Zimmer)

'X   '

Also werden als erstes alle Leerzeichen vor und hinter einem Eintrag gelöscht, und anschliessend die "X" ersetzt

In [6]:
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Für alle Einträge, die Strings sind, wird die Funktion .strip() aufgerufen, was Leerzeichen vor und hinter dem String entfernt
data_mietpreis = data.replace('X', np.nan)  # Alle "X" werden mit np.nan ersetzt
data_mietpreis  # print

,Total,1,2,3,4,5,6+
Kanton,,,,,,,
Schweiz,1362,794.0,1101,1317,1574,1910,2360
Zürich,1592,951.0,1339,1538,1835,2253,2907
Bern,1223,702.0,988,1187,1417,1691,2079
Luzern,1368,757.0,1073,1315,1509,1849,2145
Uri,1210,NaN,846,1166,1329,1541,1745
Schwyz,1552,744.0,1145,1444,1771,2092,2738
Obwalden,1347,593.0,922,1226,1534,1829,1880
Nidwalden,1480,663.0,1002,1394,1682,2060,1938
Glarus,1169,537.0,939,1088,1314,1592,1528


Die resultierende Tabelle wird zum Weiterverwenden in Part 2 des Vortrags als csv gespeichert:

In [7]:
data_mietpreis.to_csv('./mietpreis.csv', index=True)

Weitere Datenquelle
===================
Als zweite Datenquelle wird die durchschnittliche Wohnungsgrösse in m2 pro Kanton und Zimmerzahl verwendet ([Link](https://www.bfs.admin.ch/bfs/de/home/statistiken/bau-wohnungswesen.assetdetail.14407221.html)).
Das Vorgehen ist dasselbe wie oben, die Parameter skiprows, skipfooter und index werden dem Excel angepasst.

In [8]:
data = pd.read_excel('Wohnflaeche.xlsx', skiprows=4, skipfooter=5, index_col=0)
data = data.rename(columns={'Unnamed: 1': 'Total'})
data.index.name = 'Kanton'
data_flaeche = data

In [9]:
data_flaeche.to_csv('./wohnflaeche.csv')